# 1. Scrape pages

In [2]:
!pip install wikipedia -q

In [24]:
import wikipedia
import json

topics = []

# Open file
with open("wiki-list.json") as topic_list:
    topics = json.load(topic_list)["Topics"][:1]

# Get wikipedia page
# Todo: make it loop through pages
contexts = wikipedia.summary(topics[0])


'Artificial intelligence (AI) is intelligence—perceiving, synthesizing, and inferring information—demonstrated by machines, as opposed to intelligence displayed by non-human animals and humans. Example tasks in which this is done include speech recognition, computer vision, translation between (natural) languages, as well as other mappings of inputs.'

# 2. Generate Questions

# 3. Filter Questions